In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.ndimage import uniform_filter1d, gaussian_filter1d
from toolkit.common.constants import *
from toolkit.lapsim.lap_sims import RunSim
from toolkit.cars.car_configuration import Car
from toolkit.lap.tracks import *

from toolkit.cars import sr_9
from toolkit.las_solvers import Octahedral_LAS, Multi_Layer_LAS, LAS
from toolkit.steady_state_solver import LS_Solver, Parachute, Iterative_Solver
from toolkit.tire_model.tire_model_utils import * # this has the tire models

In [ ]:
# import a track you want to run
track = get_MIS_2017_AX_3_track(50)
# track = get_Crows_2022(50)

In [ ]:
# import SR9 for instance
car = Car()
# car = sr_9()
car.set_tire(H_R20_18X6_7)
# solver = LS_Solver()
solver = Iterative_Solver()
# las = Octahedral_LAS(solver=solver)
las = Multi_Layer_LAS(solver=solver)
las.add_layers = 1
# car.debug = True

In [ ]:
# set some settings
# car.debug = True
v_average = 15
mu = 0.65 # mu correction factor, a value of 1 means the road is sandpaper and the actual value should be something lower but im kinda just setting this value to overfit atm
target = 0.0001

In [ ]:
# make the simulation object
sim = RunSim(track, car, las)

In [ ]:
# run the simulation
sim.simulate(sim_type='qts', v_average=v_average, mu=mu, convergence_target=target, bins=20)

In [ ]:
sim.plot_vs(distance=True, delta_beta_est=False, fz=False, separate_angles=True, yaw_acc=True, separate_acc=True, power_draw=False)

In [ ]:
fig = go.Figure()
find_zeros = np.where(sim.lon == 0)[0]
print(f"Number of zeros: {find_zeros}")

# plot the count of the number of times each point was solved
fig.add_trace(go.Scatter(x=np.arange(len(sim.count)), y=sim.count, mode='lines', marker=dict(size=2, color='red')))
fig.add_trace(go.Scatter(x=np.arange(len(sim.count)), y=sim.last_changed, mode='lines', marker=dict(size=2, color='blue')))
fig.add_trace(go.Scatter(x=find_zeros, y=np.zeros(len(find_zeros)), mode='markers', marker=dict(size=2, color='green')))
fig.update_layout(template="plotly_dark", title_text="Number of Points Solved")
fig.show()



In [ ]:
sim.plot_vs_vel()

In [ ]:
sim.plot_mmd3d()

In [ ]:
sim.plot_gg_vel()

In [ ]:
sim.plot_mmd_vel()

In [ ]:
sim.map_plot()

In [ ]:
px.set_mapbox_access_token(open(".mapbox_token").read())
fig = px.scatter_mapbox(lat=sim.track.lat_ss, lon=sim.track.lon_ss, color=np.abs(sim.track.real_beta), color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
fig.show()

In [ ]:
px.set_mapbox_access_token(open(".mapbox_token").read())
fig = px.scatter_mapbox(lat=sim.track.lat_ss, lon=sim.track.lon_ss, color=sim.omega, color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
fig.show()

In [ ]:
px.set_mapbox_access_token(open(".mapbox_token").read())
fig = px.scatter_mapbox(lat=sim.track.lat_ss, lon=sim.track.lon_ss, color=sim.track.k, color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
fig.show()

In [ ]:
px.set_mapbox_access_token(open(".mapbox_token").read())
fig = px.scatter_mapbox(lat=sim.track.lat_ss, lon=sim.track.lon_ss, color=sim.track.u, color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
fig.show()

In [ ]:
px.set_mapbox_access_token(open(".mapbox_token").read())
fig = px.scatter_mapbox(lat=sim.track.lat_ss, lon=sim.track.lon_ss, color=sim.track.angle, color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
fig.show()

In [ ]:
# plot the results, this plots them all together but you can also plot them individually
sim.plot()

In [ ]:
print(sim.car.vels)

In [ ]:
print(sim.car.vps)

In [ ]:
sim.plot_LAS_lims()

In [ ]:
sim.plot_convergence()